In [1]:
# import spark and initialize session
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('MSD').getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1606774301209_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("IPython")
sc.install_pypi_package("seaborn")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/bf/4c/cb7da76f3a5e077e545f9cf8575b8f488a4e8ad60490838f89c5cdd5bb57/pandas-1.1.4-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/30/f2/10c822cb0ca5ebec58bd1892187bc3e3db64a867ac26531c6204663fc218/matplotlib-3.3.3-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8a/bb/488841f56197b13700afd5658fc279a2025a39e22449b7cf29864669b15d/pyparsing-2.4.7-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/af/fa/c1302a26d5e1a17fa8e10e43417b6cf038b0648c4b79fcf2302a4a0c5d30/Pillow-8.0.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
  Using ca

In [3]:
# import libraries
import numpy as np
import csv
import pandas as pd
import seaborn as sns
from IPython.display import display
import matplotlib
from matplotlib import pyplot as plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# read from s3
df = spark.read.csv("s3a://msd-bucket-10605/msd-data/*.csv", header=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# rename
col_names = ['analyzer_version', 'artist_7digitalid', 'artist_familiarity', 'artist_hotttnesss', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_mbid', 'artist_name', 'artist_playmeid', 'genre', 'idx_artist_terms', 'idx_similar_artists', 'release', 'release_7digitalid', 'song_hotttnesss', 'song_id', 'title', 'track_7digitalid', 'analysis_sample_rate', 'audio_md5', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'idx_bars_confidence', 'idx_bars_start', 'idx_beats_confidence', 'idx_beats_start', 'idx_sections_confidence', 'idx_sections_start', 'idx_segments_confidence', 'idx_segments_loudness_max', 'idx_segments_loudness_max_time', 'idx_segments_loudness_start', 'idx_segments_pitches', 'idx_segments_start', 'idx_segments_timbre', 'idx_tatums_confidence', 'idx_tatums_start', 'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo', 'time_signature', 'time_signature_confidence', 'track_id', 'idx_artist_mbtags', 'year']
df = df.toDF(*col_names)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

999266

In [ ]:
df.printSchema()

In [7]:
# drop columns
drop_cols = ['analyzer_version', 'artist_7digitalid', 'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude', 'artist_mbid', 'artist_playmeid', 'genre', 'release_7digitalid', 'song_id', 'track_7digitalid', 'analysis_sample_rate', 'audio_md5', 'idx_bars_confidence', 'idx_beats_confidence', 'idx_sections_confidence', 'idx_segments_confidence', 'idx_segments_loudness_max_time', 'idx_segments_loudness_start', 'idx_segments_pitches', 'idx_segments_start', 'idx_segments_timbre', 'idx_tatums_confidence', 'track_id']
df = df.drop(*drop_cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
print(df.columns)
print(len(df.columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist_familiarity', 'artist_hotttnesss', 'artist_name', 'idx_artist_terms', 'idx_similar_artists', 'release', 'song_hotttnesss', 'title', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'idx_bars_start', 'idx_beats_start', 'idx_sections_start', 'idx_segments_loudness_max', 'idx_tatums_start', 'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo', 'time_signature', 'time_signature_confidence', 'idx_artist_mbtags', 'year']
28

In [9]:
from pyspark.sql.functions import col

str_columns = ['artist_name', 'release', 'title']
float_columns = [col for col in df.columns if col not in str_columns]
print(float_columns)
for col_name in float_columns:
    df = df.withColumn(col_name, col(col_name).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist_familiarity', 'artist_hotttnesss', 'idx_artist_terms', 'idx_similar_artists', 'song_hotttnesss', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'idx_bars_start', 'idx_beats_start', 'idx_sections_start', 'idx_segments_loudness_max', 'idx_tatums_start', 'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo', 'time_signature', 'time_signature_confidence', 'idx_artist_mbtags', 'year']

In [10]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_familiarity: float (nullable = true)
 |-- artist_hotttnesss: float (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- idx_artist_terms: float (nullable = true)
 |-- idx_similar_artists: float (nullable = true)
 |-- release: string (nullable = true)
 |-- song_hotttnesss: float (nullable = true)
 |-- title: string (nullable = true)
 |-- danceability: float (nullable = true)
 |-- duration: float (nullable = true)
 |-- end_of_fade_in: float (nullable = true)
 |-- energy: float (nullable = true)
 |-- idx_bars_start: float (nullable = true)
 |-- idx_beats_start: float (nullable = true)
 |-- idx_sections_start: float (nullable = true)
 |-- idx_segments_loudness_max: float (nullable = true)
 |-- idx_tatums_start: float (nullable = true)
 |-- key: float (nullable = true)
 |-- key_confidence: float (nullable = true)
 |-- loudness: float (nullable = true)
 |-- mode: float (nullable = true)
 |-- mode_confidence: float (nullable = true)
 |-- start_of_fade_out: float 

In [64]:
# # has valid year
# df_yr = df.filter(df['year'] > 0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_model = df.filter(df['song_hotttnesss'] >= 0)
df_model = df_model.filter(df_model['song_hotttnesss'] <= 1)
df_model = df_model.filter(df_model['year'] > 0)
# df_lr = df_no_yr.filter(df_no_yr['song_hotttnesss'] > 0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_model.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

345843

In [142]:
df_model.describe()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[summary: string, artist_familiarity: string, artist_hotttnesss: string, artist_name: string, idx_artist_terms: string, idx_similar_artists: string, release: string, song_hotttnesss: string, title: string, danceability: string, duration: string, end_of_fade_in: string, energy: string, idx_bars_start: string, idx_beats_start: string, idx_sections_start: string, idx_segments_loudness_max: string, idx_tatums_start: string, key: string, key_confidence: string, loudness: string, mode: string, mode_confidence: string, start_of_fade_out: string, tempo: string, time_signature: string, time_signature_confidence: string, idx_artist_mbtags: string, year: string]

In [13]:
# df_model = df_model.drop('year')
# year_mean = df_model.filter(df_model['year'] > 0).select(avg('year')).collect()[0]['avg(year)']
# year_mean
# df_model = df_model.withColumn('year', when(col('year') == 0.0, year_mean).otherwise(col('year')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1999.5491509152996

In [13]:
# baseline
df_base = df_model
artist_hotness_mean = df_base.select(avg('artist_hotttnesss')).collect()[0]['avg(artist_hotttnesss)']
artist_hotness_mean

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.42233044703029526

In [14]:
df_base = df_base.withColumn("prediction", lit(0))
df_base = df_base.withColumn("prediction", when(col("artist_hotttnesss") > artist_hotness_mean, 1).otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
song_hotness_mean = df_base.select(avg('song_hotttnesss')).collect()[0]['avg(song_hotttnesss)']
song_hotness_mean

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4306739799357913

In [16]:
df_base = df_base.withColumn("isPopular", lit(0))
df_base = df_base.withColumn("isPopular",when(col("song_hotttnesss") > song_hotness_mean, 1).otherwise(0))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
_, baseline_test = df_base.randomSplit([0.8, 0.2], seed=10000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
base_accuracy = baseline_test.filter(baseline_test.isPopular == baseline_test.prediction).count()/float(baseline_test.count())
print("Baseline Accuracy: ", base_accuracy)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Baseline Accuracy:  0.6620129587122643

In [19]:
# Logistic Regression
df_lr = df_model
song_hotness_mean = df_lr.select(avg('song_hotttnesss')).collect()[0]['avg(song_hotttnesss)']
song_hotness_mean

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.4306739799357913

In [20]:
df_lr = df_lr.withColumn("isPopular", lit(0))
df_lr = df_lr.withColumn("isPopular",when(col("song_hotttnesss") > song_hotness_mean, 1).otherwise(0))
df_lr = df_lr.fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# assembly all features and normalize each feature
from pyspark.ml.feature import VectorAssembler, StandardScaler
feat_columns = ['artist_familiarity', 'artist_hotttnesss', 'idx_artist_terms', 'idx_similar_artists', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'idx_bars_start', 'idx_beats_start', 'idx_sections_start', 'idx_segments_loudness_max', 'idx_tatums_start', 'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo', 'time_signature', 'time_signature_confidence', 'idx_artist_mbtags', 'year']
# feat_columns = ['artist_familiarity', 'artist_hotttnesss', 'duration', 'loudness', 'start_of_fade_out']
assembler = VectorAssembler().setInputCols(feat_columns).setOutputCol('features')
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
assembler_df = assembler.transform(df_lr)
scalerModel = scaler.fit(assembler_df)
scaler_df = scalerModel.transform(assembler_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# can be used for different models
train, test = scaler_df.randomSplit([0.8, 0.2], seed=10000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# LogisticRegression training
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol='scaledFeatures', labelCol='isPopular', maxIter=30)
lrModel = lr.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# LR Model Evaluation
predictions = lrModel.transform(test)
lr_accuracy = predictions.filter(predictions.isPopular == predictions.prediction).count()/float(predictions.count())
print("LR Accuracy: ", lr_accuracy)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LR Accuracy:  0.702574309207659

In [25]:
print(lrModel.intercept)
print(lrModel.coefficients)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.20401905156529598
[0.6367514895522045,0.554898116295588,0.0,0.0,0.0,-0.004108259142836956,0.023855556463562858,0.0,0.0,0.0,0.0,0.0,0.0,-0.01852672841435136,-0.03001622479907452,0.19894158885872126,-0.053836020583206586,0.03333536877858256,-0.01092252832448459,0.02506119249924211,-0.00850558052588035,0.004188083904888363,0.0,-0.025235715554321556]

In [27]:
# Random Forest Classifier
from pyspark.ml.classification import RandomForestClassifier
# from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="isPopular", featuresCol="scaledFeatures", maxDepth=20, numTrees=20)
# Train model.  This also runs the indexers.
rfModel = rf.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Make predictions.
rf_predictions = rfModel.transform(test)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="isPopular", predictionCol="prediction", metricName="accuracy")
rf_accuracy = evaluator.evaluate(rf_predictions)
print("Accuracy = %g" % (rf_accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.71148

In [29]:
# treeModel = model.stages[2]
# summary only
feat_importances = list(rfModel.featureImportances)
# feat_columns = ['artist_familiarity', 'artist_hotttnesss', 'idx_artist_terms', 'danceability', 'duration', 'end_of_fade_in', 'energy', 'idx_bars_start', 'idx_beats_start', 'idx_sections_start', 'idx_segments_loudness_max', 'idx_tatums_start', 'key', 'key_confidence', 'loudness', 'mode', 'mode_confidence', 'start_of_fade_out', 'tempo', 'time_signature', 'time_signature_confidence']
feat_importances

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.22378014016878495, 0.18285441511225453, 0.0, 0.0, 0.0, 0.051374181441056425, 0.049792223578997645, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.04508482549246076, 0.05815775185520024, 0.07059872971824574, 0.00925279923856773, 0.058925443579135116, 0.050923635119145, 0.06126663196012092, 0.021221029359270387, 0.048865409240860976, 0.0, 0.06790278413589926]

In [30]:
# gradient boosting
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="isPopular", featuresCol="scaledFeatures", maxIter=30)
gbModel = gbt.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
gb_predictions = gbModel.transform(test)
gb_evaluator = MulticlassClassificationEvaluator(labelCol="isPopular", predictionCol="prediction", metricName="accuracy")
gb_accuracy = gb_evaluator.evaluate(gb_predictions)
print("Accuracy = %g" % (gb_accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.711828

In [32]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

layers = [24, 48, 20, 2]
mlp = MultilayerPerceptronClassifier(featuresCol="scaledFeatures", labelCol="isPopular", maxIter=100, layers=layers, blockSize=512)
mlpModel = mlp.fit(train.select('scaledFeatures', 'isPopular'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
mlp_result = mlpModel.transform(test.select('scaledFeatures', 'isPopular'))
mlp_result = mlp_result.select("prediction", "isPopular")
mlp_evaluator = MulticlassClassificationEvaluator(labelCol="isPopular", predictionCol="prediction", metricName="accuracy")
mlp_accuracy = mlp_evaluator.evaluate(mlp_result)
print("Accuracy = %g" % (mlp_accuracy))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy = 0.702284

In [35]:
# SVM
from pyspark.ml.classification import LinearSVC
svm = LinearSVC(featuresCol="scaledFeatures", labelCol="isPopular", maxIter=30)
svmModel = svm.fit(train)


VBox()

An error was encountered:
Session 5 did not reach idle status in time. Current status is busy.


In [36]:
svm_result = svmModel.transform(test)
svm_evaluator = MulticlassClassificationEvaluator(labelCol="isPopular", predictionCol="prediction", metricName="accuracy")
svm_accuracy = svm_evaluator.evaluate(mlp_result)
print("Accuracy = %g" % (svm_accuracy))

VBox()

An error was encountered:
Session 5 did not reach idle status in time. Current status is busy.


In [38]:
accuracies = [base_accuracy, lr_accuracy, rf_accuracy, gb_accuracy, mlp_accuracy]
accuracies

VBox()

An error was encountered:
Session 5 did not reach idle status in time. Current status is busy.


In [39]:
# ROC
print(svmModel.summary.roc.select('FPR').collect())
print(svmModel.summary.roc.select('TPR').collect())
print(gbModel.summary.roc.select('FPR').collect())
print(gbModel.summary.roc.select('TPR').collect())

VBox()

An error was encountered:
Session 5 did not reach idle status in time. Current status is busy.
